In [1]:
import respy as rp
import numpy as np
from estimagic.optimization.optimize import maximize

### Working step 1: Model estimation via Maximum Likelihood

* We estimate a model by maximum likelihood using all true values as the starting point. Note that this model does not include $\eta$ and none of the values is fixed (different to Working step 3). We use a gradient-free optimizer.

Simulate the model at starting values.

In [2]:
params, options = rp.get_example_model("kw_94_one", with_data=False)
simulate = rp.get_simulate_func(params, options)
df = simulate(params)

Get the criterion function and output the value of the MLE at given parameter values.

In [3]:
crit_func = rp.get_crit_func(params, options, df)
crit_func(params)

-380.4868525056946

In [4]:
# Note: maxfun is for scipy optimizers
# Note: maxeval is for nlopt optimizer
#algo_options = {"maxeval": 100}
#algo_options={"maxfun": 1}
#algo_name = "nlopt_bobyqa" (gradient-free)
#algo_name = "scipy_L-BFGS-B" (gradient-based)

In [5]:
# Assemble the parameter constraints for kw_94_one
# Note: # constr = rp.get_parameter_constraints("kw_94_one") # is not working
constr = [
    {"loc": "shocks_sdcorr", "type": "sdcorr"},
    {"loc": "lagged_choice_1_edu", "type": "fixed"},
    {"loc": "initial_exp_edu_10", "type": "fixed"},
    {"loc": "maximum_exp", "type": "fixed"},
]

In [6]:
constr

[{'loc': 'shocks_sdcorr', 'type': 'sdcorr'},
 {'loc': 'lagged_choice_1_edu', 'type': 'fixed'},
 {'loc': 'initial_exp_edu_10', 'type': 'fixed'},
 {'loc': 'maximum_exp', 'type': 'fixed'}]

In [7]:
results, params_rslt = maximize(
    crit_func, 
    params, 
    "nlopt_bobyqa", 
    algo_options={"maxeval": 2},  # for the first tries we set the evaluations to a low number
    constraints=constr,
    dashboard=False)

In [8]:
#params_rslt

### Working step 2: Ambiguity perturbation

We will disturb $\eta$ and subsequently estimate it to check whether the same values come out again.

More specifically:
* we will first set $\eta$ so the values presented at p.17 in the paper, i.e. baseline (0.01) and high (0.02)
* we will simulate the model with the set values
* we will perturb our starting values in the estimation and analyze whether they return to their initial values

In [9]:
# Check the parameter combinations
eta_perturbed = {
    "true": [],
    "estimated": [],
    "yos": []
}

eta_perturbed = {
    "true": [],
    "estimated": []
}

In [10]:
constr = [{"query": "category != 'eta'", "type": "fixed"}]

In [11]:
constr

[{'query': "category != 'eta'", 'type': 'fixed'}]

In [12]:
eta_baseline = 0.01
eta_high = 0.02
# To do: in the end we will have a wrapper that works for simulation with any given eta

In [13]:
# Decide for an eta
params_base, options = rp.get_example_model("kw_94_one", with_data=False)
params_base.loc[("eta", "eta"), :] = (eta_baseline, "eta value " + str(eta_baseline))
simulate = rp.get_simulate_func(params, options)

In [14]:
for eta in np.linspace(0, 0.25, 11):
    
    #params, options = rp.get_example_model("kw_94_one", with_data=False)
    #simulate = rp.get_simulate_func(params, options)
    
    #params.loc[("eta", "eta"), :] = (eta, "iteration with " + str(eta))
    params_base.loc[("eta", "eta"), :] = eta
    #df = simulate(params) # We want only to change the starting values, but not the simulated "data"
    #stat = df.groupby("Identifier")["Experience_Edu"].max().mean()
    
    #params["group"] = params.index.get_level_values('category')
    # need above only if we want dashboard output
    
    crit_func = rp.get_crit_func(params, options, df)
    # crit_func(params) # no need to output the value here
    # constr = rp.get_parameter_constraints("kw_94_one")
    # just take constraints from above with all other except eta fixed

    results, params = maximize(
        crit_func, 
        params_base,
        "nlopt_bobyqa",
        algo_options={"maxeval": 2},
        # In the "real world simulation" we will set this to a much higher number
        constraints=constr,
        dashboard=False
    )
    
    eta_perturbed["true"].append(eta_baseline)
    eta_perturbed["estimated"].append(params.loc["eta", "value"][0])
    #eta_perturbed["yos"].append(stat)
    
    # To do: write a wrapper for this thing

### Working step 3: Risk-only model

We estimate a model with *risk-only*. This means we have $\eta = 0$ fixed. 
We want to see how different the likelihood is at the estimated parameter values.
We treat the risk-parameter as only unknown parameter.

In [32]:
params_base.loc[("eta", "eta"), :] = 0
constr_delta = [{"query": "category != delta", "type": "fixed"}]

In [33]:
crit_func = rp.get_crit_func(params, options, df)
crit_func(params)

-380.4868525056946

In [34]:
params_base

value  \
category                name                                   
delta                   delta                         0.9500   
wage_a                  constant                      9.2100   
                        exp_edu                       0.0380   
                        exp_a                         0.0330   
                        exp_a_square                 -0.0005   
                        exp_b                         0.0000   
                        exp_b_square                  0.0000   
wage_b                  constant                      8.4800   
                        exp_edu                       0.0700   
                        exp_b                         0.0670   
                        exp_b_square                 -0.0010   
                        exp_a                         0.0220   
                        exp_a_square                 -0.0005   
nonpec_edu              constant                      0.0000   
                        at_least_twelve_exp_edu       0.0000   
                        not_edu_last_period       -4000.0000   
nonpec_home             constant                  17750.0000   
shocks_sdcorr           sd_a                          0.2000   
                        sd_b                          0.2500   
                        sd_edu                     1500.0000   
                        sd_home                    1500.0000   
                        corr_b_a                      0.0000   
                        corr_edu_a                    0.0000   
                        corr_edu_b                    0.0000   
                        corr_home_a                   0.0000   
                        corr_home_b                   0.0000   
                        corr_home_edu                 0.0000   
lagged_choice_1_edu     probability                   1.0000   
initial_exp_edu_10      probability                   1.0000   
maximum_exp             edu                          20.0000   
inadmissibility_penalty inadmissibility_penalty -400000.0000   
eta                     eta                           0.0000   

                                                                                           comment  
category                name                                                                        
delta                   delta                                                      discount factor  
wage_a                  constant                                               log of rental price  
                        exp_edu                          return to an additional year of schooling  
                        exp_a                                     return to same sector experience  
                        exp_a_square                   return to same sector, quadratic experience  
                        exp_b                                    return to other sector experience  
                        exp_b_square                  return to other sector, quadratic experience  
wage_b                  constant                                               log of rental price  
                        exp_edu                          return to an additional year of schooling  
                        exp_b                                     return to same sector experience  
                        exp_b_square                   return to same sector, quadratic experience  
                        exp_a                                    return to other sector experience  
                        exp_a_square                  return to other sector, quadratic experience  
nonpec_edu              constant                            constant reward for choosing education  
                        at_least_twelve_exp_edu        reward for going to college (tuition, etc.)  
                        not_edu_last_period                        reward for going back to school  
nonpec_home             constant                         con

In [35]:
results, params_rslt = maximize(
    crit_func, 
    params_base,
    "nlopt_bobyqa",
    algo_options={"maxeval": 2},
    constraints=constr_delta,
    dashboard=False
)

UndefinedVariableError: name 'delta' is not defined

In [ ]:
params_rslt

### Summary of working steps

* We estimated a model by maximum likelihood using all true values as the starting point.

* We estimated a model with only $\eta$ perturbed.

* We estimated a model with risk-only, $\eta = 0$ fixed, to see how different the likelihood is at the estimated parameter values.